In [1]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.path.join(os.getcwd(), ".."), ".."))
sys.path.append(parent_dir)
from pathlib import Path

import numpy as np
from data_generation.models.tech_substitution import TechnologySubstitution, NumericalSolver
from data_generation.simulations.grid import Grid, fractional_transformation
from data_generation.simulations.simulator import Simulator

In [2]:
OUTPUT_DIR = Path.cwd().parent.parent / "datasets" / "results"
OUTPUT_DIR

PosixPath('/home/julian/tuberlin/projects/mdp-world-model/datasets/results')

In [3]:
def test_simulation_storage(output_dir):
    """Test simulating and storing technology substitution trajectory
    
    Args:
        output_dir (str or Path): Absolute path to output directory
    """
    # Setup paths
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Setup simulation
    bounds = [(0, np.inf), (0, np.inf)]
    resolution = [7, 7]
    transformations = [fractional_transformation(3), fractional_transformation(3)]
    
    # Create objects
    grid = Grid(bounds, resolution, transformations)
    model = TechnologySubstitution()
    solver = NumericalSolver(model)
    simulator = Simulator(grid, model, solver)
    
    # Run simulation
    control = 0.5
    results = simulator.simulate(
        control=control,
        delta_t=0.1,
        num_samples_per_cell=3,  
        num_steps=50,
        save_result=True
    )
    
    # Store results
    db_path = output_path / 'simulation_results_test.db'
    simulator.store_results_to_sqlite(db_path)
    
    print(f"Stored {len(results)} rows of simulation data in {db_path}")
    return simulator

In [4]:
simulator = test_simulation_storage(OUTPUT_DIR)

# Print info about stored data
print("\nConfigs data:")
print(simulator.configs.head())
print("\nResults data:")
print(simulator.results.head())

Simulation complete:
- 147 samples × 50 timesteps = 7350 total rows
- State dimensions: 2
- Control dimensions: 1
Saved results and config.
Stored 7350 rows of simulation data in /home/julian/tuberlin/projects/mdp-world-model/datasets/results/simulation_results_test.db

Configs data:
            run_id                                           metadata
0  20250207_163214  {"configurations": {"grid": {"bounds": [[0, In...

Results data:
            run_id trajectory_id   t0   t1        x0        x1   c0        y0  \
0  20250207_163214         0-0_0  0.0  0.1  0.125069  0.408799  0.5  0.125123   
1  20250207_163214         0-0_0  0.1  0.2  0.125123  0.445972  0.5  0.125169   
2  20250207_163214         0-0_0  0.2  0.3  0.125169  0.485150  0.5  0.125209   
3  20250207_163214         0-0_0  0.3  0.4  0.125209  0.526236  0.5  0.125242   
4  20250207_163214         0-0_0  0.4  0.5  0.125242  0.569136  0.5  0.125271   

         y1  
0  0.445972  
1  0.485150  
2  0.526236  
3  0.569136  
4  